# Run.c

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/karpathy/llama2.c/blob/master/run.ipynb)

More details can be found in the [README.md](README.md) .

In [1]:
#@title Clone Project
!rm -rf tem-force
!git clone https://github.com/yyj6666667/tem-force
!ls
%cd tem-force

Cloning into 'tem-force'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 107 (delta 36), reused 91 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 1.08 MiB | 4.11 MiB/s, done.
Resolving deltas: 100% (36/36), done.
sample_data  tem-force
/content/tem-force


In [2]:
#@title Pick Your Model

#@markdown Choose model
model = "stories42M" #@param ["stories15M", "stories42M", "stories110M"]

download_url = ""

if(model == "stories15M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin"
if(model == "stories42M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin"
if(model == "stories110M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin"

print(f"download_url: {download_url}")

!wget $download_url

model_file = model + ".bin"

download_url: https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin
--2026-01-10 14:51:35--  https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin
Resolving huggingface.co (huggingface.co)... 3.168.132.91, 3.168.132.38, 3.168.132.40, ...
Connecting to huggingface.co (huggingface.co)|3.168.132.91|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/64c19eb0275dc8cd7f170b8f/20ab6952f8b77338484a45f788d50ed30dfda285f9954758b75d7deefd120d26?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27stories42M.bin%3B+filename%3D%22stories42M.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1768060295&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY4MDYwMjk1fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjRjMTllYjAyNzVkYzhjZDdmMTcwYjhmLzIwYWI2OTUyZjhiNzczMzg0ODRhNDVmNzg4ZDUwZWQzMGRmZGEyODVmOTk1NDc1OGI3NWQ3ZGVlZmQ

In [42]:
#CPU;
!make runfast

gcc -Ofast -o run run.c -lm
gcc -Ofast -o runq runq.c -lm


若使用 CUDA 优化，需执行nvcc -arch=compute_35 -O3 -std=c++17 -o run run.c -lm命令编译代码；若使用了 cuBLAS 库，则执行nvcc -arch=compute_35 -O3 -std=c++17 -o run run.c -lm -L/usr/local/lib64 -lcublas编译代码；若采用 AVX、OpenMP 等其他优化方法，则直接执行make run即可。

In [3]:
#GPU
#!nvcc -O3 -o run run.cu -lcudart -lm

!nvcc -O3 -std=c++14 -arch=sm_75 -o run run_version2.cu

run_version2.cu:117:48: warning: backslash and newline separated by space
  117 |     #define CP_HOST2DEVICE(dst, src, len) do { \
      |                                                 
run_version2.cu:118:48: warning: backslash and newline separated by space
  118 |         int bytes = (len) * sizeof(float);     \
      |                                                 
run_version2.cu:119:36: warning: backslash and newline separated by space
  119 |         cudaMalloc(&(dst), bytes); \
      |                                     
run_version2.cu:120:66: warning: backslash and newline separated by space
  120 |         cudaMemcpy((dst), (src), bytes, cudaMemcpyHostToDevice); \
      |                                                                   
run_version2.cu(641): warning #2464-D: conversion from a string literal to "char *" is deprecated
          int dummy_prefix = str_lookup(" ", t->sorted_vocab, t->vocab_size);
                                        ^

Remark: The warni

In [7]:

!apt-get install -y cuda-nsight-systems-12-2 >/dev/null || echo "nsys is installed"

!nvprof --version
!nsys --version


W: Operation was interrupted before it could finish
nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2024 NVIDIA Corporation
Release version 12.5.82 (21)
NVIDIA Nsight Systems version 2023.2.3.1004-33186433v0


In [8]:
#备选方案nvprof

# 2. 运行 nvprof
!nvprof --print-gpu-summary ./run stories42M.bin  2> nvprof_output.txt
# 步骤 1: 提取 GPU activities 部分
!cat nvprof_output.txt

One day, a boy named Tim wanted to paint. He had a big box of colors and a big paper. He was very happy. He wanted to show his mom the big drawing he made. Tim ran to his mom and said, "Look, Mom! I made a big picture!"
Mom looked at the big picture and smiled. "Wow, Tim! That is a great picture!" she said. Tim felt proud of his work. He wanted to show it to his friend, Sam. So, he took his big box of colors and went to Sam's house.
When Tim got to Sam's house, he saw that his big box of colors was open. He felt a little guilty because he wanted to paint with his big box of colors. He looked at Sam and said, "I'm sorry, Sam. I left my big box open. Can I use the colors now?"
Sam smiled and said, "Of course, Tim! Let's make new pictures together." So, Tim and Sam opened the big box of colors and started to paint. They had a lot of fun and made a beautiful picture together.
==2990== NVPROF is profiling process 2990, command: ./run stories42M.bin
achieved tok/s: 159.703504
==2990== Profil

In [9]:
#@title Generate Stories

# Generate args
max_token = 256 #@param {type:"slider", min:32, max:1024, step:32}
temperature = 0.8 #@param {type:"slider", min:0.0, max:1, step:0.05}
top_p = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
prompt = "One day, Lily met a Shoggoth" #@param {type:"string"}

print(f"model: {model_file}, max_token: {max_token}, temperature: {temperature}, top_p: {top_p}, prompt: {prompt}")
print(f"----------------------------\n")

cmd = f'./run {model_file} -t {temperature} -p {top_p} -n {max_token} -i "{prompt}"'
!{cmd}

model: stories42M.bin, max_token: 256, temperature: 0.8, top_p: 0.9, prompt: One day, Lily met a Shoggoth
----------------------------

One day, Lily met a Shoggoth. He was very ugly and he was very scared. Lily asked him why he was so scared. He said he was afraid of the dark and all the other scary things he could do.
Lily smiled and said that she would protect him. She said she would always be there to help him.
The Shogg smiled and said he was grateful. He asked Lily to stay with him until the sun went down. Lily agreed and kept him safe.
That night, Lily and the Shogg slept peacefully. The Shogg was so happy that he was no longer scared.
In the morning, Lily and the Shoggmy woke up to find a surprise. The Shogggor had brought all of his friends together and they had a big party to celebrate.
Lily was so happy and relieved that the Shogggles were no longer scared. She smiled and knew that the Shogggles were now safe. She was glad that she had been there to protect them all.
achieve

In [10]:



# 详细采集（包含 CUDA、cuBLAS、cuDNN、NVTX、OS 运行时）
!nsys profile -t cuda,cublas,cudnn,nvtx,osrt --stats=true -o yyj_report ./run stories42M.bin


Once upon a time, there was a little boy named Timmy. Timmy loved to play with his toys, but his room was very messy. His mom said, "Timmy, you need to clean your room." Timmy didn't want to clean his room, so he left his toys on the floor and went to bed.
The next day, Timmy wanted to go to the park with his mom. But he couldn't find his shoes because his room was so messy. His mom said, "Timmy, your room is so filthy! We need to find your shoes." Timmy felt sad because he couldn't find his shoes and he couldn't go to the park with his mom.
Timmy's mom said, "Let's clean your room and you can see your shoes in the closet." Timmy helped his mom clean his room and he found his shoes. He put them on and they fit perfectly. Timmy was happy because he could go to the park with his clean shoes. The end.
achieved tok/s: 234.439834
Generating '/tmp/nsys-report-4d8c.qdstrm'
[1/8] [========================100%] yyj_report.nsys-rep
[2/8] [========================100%] yyj_report.sqlite
[3/8] Exe